In [1]:
pip install scx>=1.0.7

Note: you may need to restart the kernel to use updated packages.


<h1>Blinky P1</h1>
<p>After years of work, your friend Robert, and you launched the brand Blink. Blink's business includes assembling, selling, and distributing a smart pet wearable device, the Blinky22. This device has multiple functions; it monitors pet activity levels, tracks health indicators, provides wellness recommendations, records veterinarian visits, tracks location, and shares this all in real-time through a mobile app. Blinky22 is waterproof and light-weight. The collar is adjustable so that any pet can wear it.</p>
<p>Blinky is manufactured in two assembly plants and it is sold in three regions. Monthly demand per region is shared in Table 1. Currently, assembly plants have no capacity restrictions and can source as many items as needed. Blink’s 3PL carrier charges a transportation cost of $0.12 per unit per mile.</p>
<p><b>Table 1: Demand in units</b></p>
<table width="90%">
<tbody>
<tr>
<td width="20%" style="text-align: center; border: 1px solid black;">Demand</td>
<td width="10%" style="text-align: center; border: 1px solid black;">Region 1</td>
<td width="10%" style="text-align: center; border: 1px solid black;">Region 2</td>
<td width="10%" style="text-align: center; border: 1px solid black;">Region 3</td>
</tr>
<tr>
<td width="15%" style="text-align: center; border: 1px solid black;">Units per month</td>
<td width="10%" style="text-align: center; border: 1px solid black;">2500</td>
<td width="10%" style="text-align: center; border: 1px solid black;">4350</td>
<td width="10%" style="text-align: center; border: 1px solid black;">3296</td>
</tr>
</tbody>
</table>
<p><b>Table 2: Distance in Miles</b></p>
<table width="90%" style="height: 76.7814px;">
<tbody>
<tr style="height: 25.5938px;">
<td width="15%" style="text-align: center; border: 1px solid black; height: 25.5938px;">Miles</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">Region 1</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">Region 2</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">Region 3</td>
</tr>
<tr style="height: 25.5938px;">
<td width="15%" style="text-align: center; border: 1px solid black; height: 25.5938px;">Assembly Plant 1</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">105</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">256</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">86</td>
</tr>
<tr style="height: 25.5938px;">
<td width="15%" style="text-align: center; border: 1px solid black; height: 25.5938px;">Assembly Plant 2</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">240</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">136</td>
<td width="10%" style="text-align: center; border: 1px solid black; height: 25.5938px;">198</td>
</tr>
</tbody>
</table>
<p></p>
<p><strong>Formulate a model using the available information. Your goal is to minimize the total transportation cost.</strong></p>
<a href="https://youtu.be/uNaJ9FcWt_s">Blinky P1 Video Walkthrough Link</a><br/>
<iframe width="560" height="315"
    src="//www.youtube.com/embed/uNaJ9FcWt_s" frameborder="0" allowfullscreen>
</iframe>

In [2]:
from scx.optimize import Model

In [3]:
transport = [
    {
        'origin_name':'A1',
        'destination_name':'R1',
        'distance': 105,
        'cost_per_mile':0.12,
    },
    {
        'origin_name':'A1',
        'destination_name':'R2',
        'distance': 256,
        'cost_per_mile':0.12,
    },
    {
        'origin_name':'A1',
        'destination_name':'R3',
        'distance': 86,
        'cost_per_mile':0.12,
    },
    {
        'origin_name':'A2',
        'destination_name':'R1',
        'distance': 240,
        'cost_per_mile':0.12,
    },
    {
        'origin_name':'A2',
        'destination_name':'R2',
        'distance': 136,
        'cost_per_mile':0.12,
    },
    {
        'origin_name':'A2',
        'destination_name':'R3',
        'distance': 198,
        'cost_per_mile':0.12,
    },
]

demand = [
    {
        'name':'R1', 
        'demand':2500
    },
    {
        'name':'R2', 
        'demand':4350
    },
    {
        'name':'R3', 
        'demand':3296
    },
]

In [4]:
for t in transport:
    # Create decision variables for each item in transport
    t['amt']=Model.variable(name=f"{t['origin_name']}__{t['destination_name']}__amt", lowBound=0)
    # Calculate the variable cost of shipping for each item in tranport
    t['cost']=t['distance']*t['cost_per_mile']

In [5]:
# Initialize the model
my_model = Model(name="Blinky22", sense='minimize')


# Add the Objective Fn
my_model.add_objective(
    fn=Model.sum([t['amt']*t['cost'] for t in transport])
)

# Add Constraints
## Demand Constraint
for d in demand:
    my_model.add_constraint(
        name=f"{d['name']}__demand",
        fn=Model.sum([t['amt'] for t in transport if t['destination_name']==d['name']]) >= d['demand'],
    )

# Solve the model
my_model.solve()

In [6]:
my_model.show_formulation()

Blinky22:
MINIMIZE
12.6*A1__R1__amt + 30.72*A1__R2__amt + 10.32*A1__R3__amt + 28.799999999999997*A2__R1__amt + 16.32*A2__R2__amt + 23.759999999999998*A2__R3__amt + 0.0
SUBJECT TO
R1__demand: A1__R1__amt + A2__R1__amt >= 2500

R2__demand: A1__R2__amt + A2__R2__amt >= 4350

R3__demand: A1__R3__amt + A2__R3__amt >= 3296

VARIABLES
A1__R1__amt Continuous
A1__R2__amt Continuous
A1__R3__amt Continuous
A2__R1__amt Continuous
A2__R2__amt Continuous
A2__R3__amt Continuous



In [7]:
# Show the outputs
my_model.show_outputs()

{'objective': 136506.72,
 'status': 'Optimal',
 'variables': {'A1__R1__amt': 2500.0,
               'A1__R2__amt': 0.0,
               'A1__R3__amt': 3296.0,
               'A2__R1__amt': 0.0,
               'A2__R2__amt': 4350.0,
               'A2__R3__amt': 0.0}}
